Bayes Factor
======

This notebook will play around with "estimating" Bayes Factor for GMM using BIC. 

We use estimating very loosely as using BIC for BF means using a uniform prior!

Using Formulation from (Wagenmakers 2007 p796): http://www.ejwagenmakers.com/2007/pValueProblems.pdf

$$BF_{01} = \frac{P(D|H_0)}{P(D|H_1)} = \exp\bigg(\frac{\text{BIC}(H_1) - \text{BIC}(H_0)}{2}\bigg) $$

In [185]:
import numpy as np
import itertools

from scipy import linalg
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from sklearn import mixture
from skfeature.function.similarity_based import SPEC
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from functools import partial
from scipy import stats


In [186]:
from sklearn.datasets import make_classification
X, y = make_classification(n_features=100, n_informative=20)

In [187]:
X1 = X[:, :80]

In [188]:
def lik_mc(X, top_n=None):
    # based on code here: 
    # http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_selection.html
    
    search_range = max(int(np.sqrt(X.shape[1])), 4)
    if top_n is None:
        top_n = search_range
    
    n_components_range = range(1, search_range)
    
    lowest_bic = np.infty
    lik = []
    pipeline = []
    kwargs = {'style': 0}
    spec_partial = partial(SPEC.spec, **kwargs)
    pipeline.append(('select top k', SelectKBest(score_func=spec_partial, k=top_n)))
    model = Pipeline(pipeline)
    
    X_sel = model.fit_transform(X, y=np.zeros(X.shape[0]))
    print(X_sel.shape)
    
    
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components)
        gmm.fit(X_sel)
        # score gives the average log likelihood
        likelihood = np.exp(gmm.score(X_sel))
        lik.append(likelihood)
    # or with np.mean..
    # this uses harmonic mean
    return stats.hmean(lik)

In [189]:
def lik_bayesfactor(lik0, lik1):
    # will calculate: P(D|H_0)/P(D|H_1) 
    #
    # from : https://mailman.ucsd.edu/pipermail/ling-r-lang-l/2013-December/000581.html
    # The BIC approximation to the Bayes Factor he advocates for BF_01 is given by exp( (BIC_1 - BIC_0)/2 ) (see page 796).
    # http://www.ejwagenmakers.com/2007/pValueProblems.pdf
    return lik0/lik1

In [190]:
def approx_bayesfactor(X0, X1, k=None):
    lik0 = lik_mc(X0, k)
    lik1 = lik_mc(X1, k)
    
    return lik0/lik1

In [191]:
top_n=10
approx_bayesfactor(X, X1, top_n)

(100, 10)
(100, 10)


8.8139424056348226

In [192]:
bic0 = lik_mc(X, top_n)
bic1 = lik_mc(X1, top_n)

(100, 10)
(100, 10)


In [193]:
bic0

5.3347493070415709e-07

In [194]:
bic1

5.7128390743999501e-08